In [1]:
import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import smooth


In [2]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time
from datetime import datetime, timedelta

fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
mask_rho = np.transpose(dataset.variables['mask_rho'], axes=(1, 0))
h =  np.transpose(dataset.variables['h'], axes=(1, 0))
h = np.where(mask_rho, h, np.nan)
angle = dataset.variables['angle'][0, 0]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')


In [3]:
df_eddies = pd.read_pickle('/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/df_eddies_processed_1462_6994.pkl')
df_eddies


,Eddy,Day,Cyc,Lon,Lat,ic,jc,xc,yc,w,q11,q12,q22,Rc,psi0,A,R,Age,Date,fname
0,1,1462,CE,156.969773,-28.461100,179,274,505.897600,1353.012251,-0.000033,1.099922,-0.193920,0.943344,104.792967,71.469206,-0.006508,69.500036,57,1994-01-02,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
1,1,1463,CE,156.993096,-28.433992,179,275,507.149717,1356.633687,-0.000030,1.066764,-0.119714,0.950849,102.305989,70.018058,-0.006690,52.250752,57,1994-01-03,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
2,1,1464,CE,157.097940,-28.381485,181,276,515.130000,1365.650728,-0.000028,1.016800,-0.154298,1.006892,115.245476,75.250400,-0.005666,67.000485,57,1994-01-04,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
3,1,1465,CE,157.094334,-28.315658,181,278,512.570772,1372.426342,-0.000033,0.896874,-0.276325,1.200119,90.907044,55.585304,-0.006726,52.001786,57,1994-01-05,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4,1,1466,CE,157.106324,-28.346479,181,277,514.728351,1369.599726,-0.000029,0.847358,-0.169788,1.214160,100.425496,58.705551,-0.005821,56.251116,57,1994-01-06,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70745,1786,6990,CE,154.083947,-37.710931,190,59,552.118058,289.998167,-0.000008,2.180373,0.030402,0.571633,64.833644,10.446315,-0.002306,47.507129,23,2009-02-20,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
70746,1786,6991,CE,154.121845,-37.531688,189,63,549.670958,309.999231,-0.000008,3.146780,0.016008,0.317867,56.081580,5.657382,-0.001799,42.013350,23,2009-02-21,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
70747,1786,6992,CE,154.003467,-37.645734,188,60,543.275124,294.416933,-0.000006,1.417053,-0.146573,0.720851,72.785024,11.906976,-0.002248,41.251686,23,2009-02-22,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
70748,1786,6993,CE,153.701377,-37.869010,184,53,524.903624,261.812814,-0.000011,0.915386,0.170795,1.124303,79.974749,14.033126,-0.002194,43.250303,23,2009-02-23,/srv/scratch/z3533156/26year_BRAN2020/outer_av...


In [7]:
sQ_col = [None] * len(df_eddies)
sRc_col = [np.nan] * len(df_eddies)

for eddy, grp in df_eddies.groupby('Eddy'):
    idx = grp.index.to_list()

    # time axis for smoothing
    ds = np.arange(len(grp))

    q11 = grp['q11'].to_numpy(float)
    q12 = grp['q12'].to_numpy(float)
    q22 = grp['q22'].to_numpy(float)

    Rcs = grp['Rc'].to_numpy(float)

    window = 5  # days
    sq11 = smooth(q11, ds, num=len(ds), window=window)
    sq12 = smooth(q12, ds, num=len(ds), window=window)
    sq22 = smooth(q22, ds, num=len(ds), window=window)
    sRc  = smooth(Rcs, ds, num=len(ds), window=window) 

    # build 2x2 matrix per time step, and write back per row
    sQ_list = [np.array([[a, b], [b, c]], float) for a, b, c in zip(sq11, sq12, sq22)]
    for k, i in enumerate(idx):
        sQ_col[i]  = sQ_list[k]
        sRc_col[i] = sRc[k]

df_eddies['sQ']  = sQ_col
df_eddies['sRc'] = sRc_col


In [19]:
import os
import matplotlib.patheffects as pe

def GIF_saver(start_day):

    save_dir = "/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/Tracking_Images"
    os.makedirs(save_dir, exist_ok=True)
    
    fnumber_old = None
    for day in df_eddies.Day.unique():

        if day >= start_day:

            df_day = df_eddies[df_eddies.Day.eq(day)]

            fname = df_day.iloc[0].fname
        
            fnumber = fname[-8:-3]
        
            if fnumber != fnumber_old:
                u_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/u_{fnumber}.npy')
                v_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/v_{fnumber}.npy')
                u_raw = np.where(np.abs(u_raw) > 1e30, np.nan, u_raw).astype(float)
                v_raw = np.where(np.abs(v_raw) > 1e30, np.nan, v_raw).astype(float)
                with nc.Dataset(fname) as dataset:
                    ocean_time = dataset.variables['ocean_time'][:].data / 86400
                fnumber_old = fnumber
        
            t_rel = np.where(ocean_time == day)[0][0]
            ut, vt = u_raw[:, :, 0, t_rel], v_raw[:, :, 0, t_rel]
            cs = np.hypot(ut, vt)
        
            fig, ax = plt.subplots(figsize=(8, 10))
            im = ax.pcolormesh(X_grid, Y_grid, cs, shading='nearest', vmin=0, vmax=2.5, cmap='Blues_r')
            fig.colorbar(im, ax=ax, label=r'Current Speed [ms$^{-1}$]')
            # spac = 1
            # ax.quiver(X_grid[spac::,::spac], Y_grid[spac::,::spac], ut[spac::,::spac], vt[spac::,::spac],
            #           color='w', width=.004, scale=50)
            
            clrs = np.where(df_day.Cyc.eq('CE'), 'c', 'r')
            ax.scatter(df_day.xc, df_day.yc, c=clrs, edgecolors='k', linewidths=0.8, s=60, zorder=10)

            for xc, yc, e, Q, Rc, cyc in zip(df_day.xc, df_day.yc, df_day.Eddy, df_day.sQ, df_day.sRc, df_day.Cyc):

                dx_ell, dy_ell = X_grid - xc, Y_grid - yc
                rho2_ell = Q[0,0]*dx_ell**2 + 2*Q[1,0]*dx_ell*dy_ell + Q[1,1]*dy_ell**2
                ax.contour(X_grid, Y_grid, rho2_ell, levels=[Rc**2/2], colors='r' if cyc=='AE' else 'c')

                ax.annotate(
                    str(e), (xc, yc),
                    textcoords='offset points', xytext=(3, 3),
                    fontsize=12, color='w', weight='bold',
                    path_effects=[pe.withStroke(linewidth=2, foreground='k')],
                    zorder=11
                )
        
            ax.set_title(f'Day {day} | {pd.Timestamp("1990-01-01") + pd.Timedelta(days=day)}')
            ax.set_aspect('equal', adjustable='datalim')
            ax.set_xlabel('x (km)')
            ax.set_ylabel('y (km)')
            ax.set_xlim(x_grid.min(), x_grid.max())
            ax.set_ylim(y_grid.min(), y_grid.max())
        
            out_path = os.path.join(save_dir, f"Day{int(day)}.png")
            fig.savefig(out_path, dpi=150, bbox_inches='tight')
            plt.close(fig)
            
GIF_saver(1462)



In [20]:
for k in range(len(df_eddies)):
    data = df_eddies.iloc[k]
    
    if np.linalg.det(np.array([[data.q11,data.q12],[data.q12,data.q22]])) < -.9:
        # print(np.linalg.det(np.array([[data.q11,data.q12],[data.q12,data.q22]])))
        print(f'Eddy{data.Eddy} Day{data.Day}')
        print(data.q11)

Eddy24 Day1694
1.1316125833246609
Eddy32 Day1558
6.438177266512443
Eddy34 Day1553
1.811735648657113
Eddy34 Day1587
0.7759680373571196
Eddy37 Day1568
2.838874206138416
Eddy50 Day1634
3.1074821453041306
Eddy59 Day1645
0.8461293430332062
Eddy63 Day1655
0.37100133669655266
Eddy65 Day1648
1.0093063429725342
Eddy65 Day1662
4.24907850700389
Eddy78 Day1697
11.28045848541853
Eddy92 Day1717
1.9903239909697346
Eddy95 Day1754
2.4303160644594493
Eddy96 Day1746
0.48289775029216314
Eddy101 Day1743
1.5319096686087799
Eddy106 Day1799
1.6349185730322005
Eddy112 Day1825
2.5851488049506033
Eddy125 Day1842
0.7166925357059175
Eddy148 Day1906
0.5159574697891043
Eddy158 Day1943
1.6578465591511542
Eddy163 Day1979
1.5636283848360446
Eddy165 Day1958
0.9812741028153668
Eddy166 Day1950
0.4764056796486165
Eddy170 Day1992
0.5848904319363871
Eddy178 Day1983
1.2217608092653305
Eddy187 Day2036
4.129582094354006
Eddy189 Day2013
1.5661762655718203
Eddy198 Day2085
2.86722482255096
Eddy201 Day2115
1.2288483856335757
Eddy25